# Tensorflowを使ってみる

まずは線形回帰で

$$y = 0.1 * x  + 0.3$$ 

を学習することをやってみる。

まずは、ランダムに`x_data`を生成して、そこから上記の数式に値をいれた`y_data`を取得する

In [1]:
import tensorflow as tf
import numpy as np

x_data = np.random.rand(100).astype("float32")
y_data = x_data * 0.1 + 0.3
print("x_data -> ", x_data[:5])
print("y_data -> ", y_data[:5])

x_data ->  [ 0.99971831  0.6914472   0.98745465  0.08627547  0.3613916 ]
y_data ->  [ 0.39997184  0.36914474  0.39874548  0.30862755  0.33613917]


次に、学習するモデルを

$$ y = W * x  + b $$

とおいて、`W`と`b`を変数とする。そのため、tensorflowのVariableとして定義する。

In [2]:
W = tf.Variable(tf.random_uniform([1], -1, 1))
b = tf.Variable(tf.zeros([1]))
print(W,b)
y = W * x_data + b
print(y)

<tensorflow.python.ops.variables.Variable object at 0x10d48b860> <tensorflow.python.ops.variables.Variable object at 0x1082abda0>
Tensor("add:0", shape=TensorShape([Dimension(100)]), dtype=float32)


`print`するとtensoflowのVariableとして定義されていることがわかる。また、`y`はTenorのインスタンスで、`"add:0"`という表現を持っていることがわかる。

さて、学習するために、コスト関数を定義する。またその後、コスト関数を最小化するための手法として最急降下法のインスタンスを、学習率0.5で生成し、コスト関数を最小化する。

In [3]:
loss = tf.reduce_mean(tf.square(y - y_data))
print("loss->", loss)
optimizer = tf.train.GradientDescentOptimizer(0.5)
print("optimizer->", optimizer)
train = optimizer.minimize(loss)
print("train->", train)

loss-> Tensor("Mean:0", shape=TensorShape([]), dtype=float32)
optimizer-> <tensorflow.python.training.gradient_descent.GradientDescentOptimizer object at 0x10d4ba438>
train-> name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_Variable/ApplyGradientDescent"
input: "^GradientDescent/update_Variable_1/ApplyGradientDescent"



すべての値を初期化する。この場合、初期化されるのは、`W`と`b`。

In [4]:
init = tf.initialize_all_variables()

tensorflowを実行するときには、Sessionを生成する

In [5]:
sess = tf.Session()
sess.run(init)

sessionはrunすることにより、Tensorのインスタンスを実行したり、Variableを表示したりすることができる。

In [6]:
for step in range(201):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(W), sess.run(b))

0 [ 0.68805414] [-0.01352155]
20 [ 0.25760698] [ 0.21832678]
40 [ 0.14198467] [ 0.27824321]
60 [ 0.11118423] [ 0.29420426]
80 [ 0.10297935] [ 0.2984561]
100 [ 0.10079366] [ 0.29958874]
120 [ 0.10021143] [ 0.29989046]
140 [ 0.10005632] [ 0.29997084]
160 [ 0.10001501] [ 0.29999223]
180 [ 0.10000399] [ 0.29999796]
200 [ 0.10000109] [ 0.29999945]


# MINISTをパーセプトロンで学習してみる

まずは、簡単なパーセプトロンで学習する。Tensorflowのチュートリアルだと、input_data.pyを使うみたいだけど、せっかくなので,Theanoのチュートリアルに合わせて、minist.pkl.gzを使う


In [7]:
import pickle, gzip
f = gzip.open('../data/mnist.pkl.gz', 'rb')
train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
print(len(train_set[0]), len(valid_set[0]), len(test_set[0]))
train_set_x, train_set_y = train_set
test_set_x, test_set_y = test_set
print(len(train_set_x[0]))
print("28x28の画像のモノクロ(白0→黒1)が１次元配列で入っている",train_set_x[:5])
print("最初のラベルは",train_set_y[0])

# ベクトルに変換する
def num_to_vec(num):
    vec = np.zeros(10)
    vec[num] = 1
    return vec

train_set_y_vec = np.array([num_to_vec(num) for num in train_set_y])
test_set_y_vec =  np.array([num_to_vec(num) for num in test_set_y])


f.close()

50000 10000 10000
784
28x28の画像のモノクロ(白0→黒1)が１次元配列で入っている [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
最初のラベルは 5


最後のアウトプットの数字は10次元のベクトル空間上の元として考える。例えば5は$[0,0,0,0,1,0,0,0,0,0]\in\mathbb{R}^{10}$で表現される。また、入力データは784次元なので、ネットワークの重み$W$は784x10の行列で表し、$\vec{evidence}$を

$$ \vec{evidence} = W \cdot \vec{x} + \vec{b}$$


とし、ソフトマックスを活性化関数として適用したものを予想値とする。$\vec{x}$の$i$要素に対して、ソフトマックスの各要素は

$$ softmax_{i} = \frac{\exp{(x_{i})}}{\sum_{j}\exp(x_{j})} $$

と定義する。これをベクトルに適用し、

$$ y = softmax(\vec{evidence}) $$

を予想値しこれで学習する。最終的には

$$ y_{predict} = argmax_{i}softmax_{i}(\vec{evidence}) $$

で予想する

In [8]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

`placefolder`はあとで、`sess.run`の時に、`feed_dict`で値を入れることができる。1変数目は型で、2変数目はサイズ。ここで、Noneとなっているのは任意の値の意味。入力の数はデータセットやミニバッチの数に酔って変わるので任意の値にしておく。

In [9]:
# これは答えを入れる
y_ = tf.placeholder(tf.float32, [None, 10])

今回のコスト関数はクロスエントロピーを使う。0~1の値なので、logはマイナスの値を取り、0の近くではマイナス無限大に発散する。そのため、値が外れていた時には、クロスエントロピーは無限大に近い値を取るので、とても学習が進む。またy_をかけているため、答えの要素のみが大きく影響することになる。

In [10]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

学習は最急降下法で行う。学習率は0.01で、クロスエントロピーを最小化する

In [11]:
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [12]:
init = tf.initialize_all_variables()

In [13]:
sess = tf.Session()
sess.run(init)

学習はミニバッチで行う。データ量が多いのため、全データでの学習をするととても時間がかかる。そのため、データを分割して学習する。

In [14]:
batch_size = 100
for step in range(1000):
    batch_index = np.random.randint(len(train_set[0])-batch_size)
    batch_x = train_set_x[batch_index:batch_index+batch_size]
    batch_y = train_set_y_vec[batch_index:batch_index+batch_size]    
    sess.run(train_step, feed_dict={x:batch_x, y_:batch_y})

これで学習が終わったので、テストデータを使って学習する。正解している数の平均を正答率とする

In [15]:
correct_prediction = tf.equal(tf.arg_max(y, 1), tf.arg_max(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print (sess.run(accuracy, feed_dict={x:test_set_x, y_:test_set_y_vec}))

0.9144


90%ぐらいの精度が出る